In [ ]:
import os
from glob import glob


In [ ]:
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

t_stds = torch.tensor(stds).cuda().half()[:,None,None]
t_means = torch.tensor(means).cuda().half()[:,None,None]

img_transforms = transforms.Compose([                        
            transforms.ToTensor(),
            transforms.Normalize(means,stds)])
 
def tensor2im(var):
     return var.mul(t_stds).add(t_means).mul(255.).clamp(0,255).permute(1,2,0)

def proc_pil_img(input_image, model):
    transformed_image = img_transforms(input_image)[None,...].cuda().half()
            
    with torch.no_grad():
        result_image = model(transformed_image)[0]; print(result_image.shape)
        output_image = tensor2im(result_image)
        output_image = output_image.detach().cpu().numpy().astype('uint8')
        output_image = PIL.Image.fromarray(output_image)
    return output_image


In [ ]:
def process(src_path, dest_path):
  in_files = sorted(glob(f'{src_path}/*'))
  for img in tqdm(in_files):
    out = f"{dest_path}/{img.split('/')[-1].split('.')[0]}.jpg"
    im = PIL.Image.open(img).resize((512, 512)).convert("RGB") 
    im = scale_by_face_size(im, target_face=300, max_res=1_500_000, max_upscale=1)
    res = proc_pil_img(im, model)
    res.save(out)


In [ ]:
# 텐서를 이미지로
import numpy as np
from PIL import Image
def tensor2im(x) -> Image.Image:
    return Image.fromarray(x.cpu().numpy().astype(np.uint8) * 255)

In [ ]:
# 이미지 다운
import requests
from io import BytesIO
from PIL import Image

def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

ip_adapter_image = download_image("https://cdn-uploads.huggingface.co/production/uploads/1668693456211-noauth.jpeg")


In [ ]:
# PIL 이미지를 그리드로 저장
def image_grid(imgs:PIL.Image, rows:int, cols:int):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid